In [137]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import spacy

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
import pickle

# Preprocessing and modeling

In [138]:
joined = pd.read_csv('joined data.csv')
joined.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id,product_id-2,product_color_id,attribute_name,attribute_value,file
0,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,LWAX0056,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,2019-10-06 15:31:31.730524+00,2019-12-19 20:40:30.786144+00,NaN,https://frame-store.com/products/les-second-me...,NaN,{},NaN,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
1,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,LWAX0056,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,2019-10-06 15:31:31.730000+00:00,2020-04-06 23:19:53.216000+00:00,2020-04-06 23:19:53.216000+00:00,https://frame-store.com/products/les-second-me...,NaN,[],185.0,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
2,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,491075,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,2019-11-11 22:22:21.664000+00:00,2020-03-25 23:24:44.823000+00:00,2020-03-23 21:06:15.953000+00:00,https://bananarepublic.gap.com/browse/product....,Everything you love about our original Madison...,[],431.0,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
3,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,491075,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,2019-11-11 22:22:21.664425+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Everything you love about our original Madison...,{},NaN,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
4,01E2C3YN4KQ36A0REWZJ89ZN73,FREDA SALVADOR,5229129,Ace Bootie,Edgy style and expert craftsmanship combine on...,Unknown,2020-03-01 22:37:32.169000+00:00,2020-04-15 21:46:03.512000+00:00,2020-03-18 23:00:31.558000+00:00,https://shop.nordstrom.com/s/freda-salvador-ac...,"True to size.\n2 1/4"" (57mm) heel (size 8.5)\n...",[],1051.0,01E2C3YN4KQ36A0REWZJ89ZN73,01E2C3YN56ZCJ8TN45V3EC8CPS,Primary Color,Blacks,initial_tags


In [139]:
joined.shape

(159013, 18)

In [140]:
joined[joined.attribute_name == 'pattern'].attribute_value.value_counts()

animal              436
logo                253
floral              209
plaid               198
stripevertical      129
geometric           127
stripehorizontal    102
graphic              94
dots                 64
abstract             57
stripe               43
colorblock           42
tiedye               33
tropical             24
camouflage           22
multiprint           19
pinstripe            18
paisley              17
houndstooth          12
monogram              3
Name: attribute_value, dtype: int64

In [141]:
joined[joined.attribute_name == 'print'].attribute_value.value_counts()

none    4
Name: attribute_value, dtype: int64

In [142]:
pattern = joined[joined.attribute_name == 'pattern']

In [143]:
## Remove unrelevant columns and then remove duplicates

pattern = pattern.drop(['mpn', 'created_at', 'updated_at', 'deleted_at', 'brand_canonical_url', 'labels', 
                        'bc_product_id', 'product_id-2', 'file'], axis = 1)
pattern = pattern.drop_duplicates()

In [144]:
pattern.head()

,product_id,brand,product_full_name,description,brand_category,details,product_color_id,attribute_name,attribute_value
10746,01DTJ8DQ5SQYBS5HX6TDH2BSJP,Hunting Season,Leather-Paneled Raffia Shoulder Bag,"Handwoven by local artisans, Hunting Season's ...",women:BAGS:SHOULDER_BAGS,"Drawstring top\nComposition: leather, raffia (...",01DTJ8DS7T1QXA88MF9T17ZKS9,pattern,colorblock
10764,01DTJ8DQ5SQYBS5HX6TDH2BSJP,Hunting Season,Leather-Paneled Raffia Shoulder Bag,"Handwoven by local artisans, Hunting Season's ...",women:BAGS:SHOULDER_BAGS,"Drawstring top\nComposition: leather, raffia (...",01DTJ8DQ64FXG2PTNDZX5QHX4D,pattern,colorblock
10780,01DTATGG2XJ7QTQC6A4PHQRYW5,Alexis,Calixte Pussy-Bow Chiffon Top,Alexis' 'Calixte' top has a vintage '60s feel ...,women:CLOTHING:TOPS,Tie fastenings at neck\nComposition: 100% poly...,01DTATGG356E9CF3V4TWB705BC,pattern,dots
10792,01DTATGSMSV7FJYTPVAYEHPD8Z,Derek Lam,Belted lace-trimmed printed silk-crepe maxi dress,Multicolored silk-crepe Slips on 100% silk; li...,Clothing / Dresses / Maxi,"Fits true to size, take your normal size \nTho...",01DTATGSN4RWS0SJYJJCWR8PA6,pattern,geometric
10827,01DVCRRBK29DWT82DRBVVY544X,Mother of Pearl,#NAME?,Black and white Lyocell Partially concealed sn...,Clothing / Tops / Blouses,"Fits true to size, take your normal size \nCut...",01DVCRRBKF3E3MTGQR5NYADG5N,pattern,geometric


In [145]:
pattern.shape

(1612, 9)

In [146]:
pattern.isnull().sum()

product_id             0
brand                  0
product_full_name      0
description          228
brand_category        42
details               82
product_color_id       0
attribute_name         0
attribute_value        0
dtype: int64

In [147]:
#replace null values with UNKNOWN_TOKEN
pattern['description'] = pattern['description'].fillna('UNKNOWN_TOKEN')
pattern['details'] = pattern['details'].fillna('UNKNOWN_TOKEN')
pattern['brand_category'] = pattern['brand_category'].fillna('UNKNOWN_TOKEN')

In [148]:
pattern.isnull().sum()

product_id           0
brand                0
product_full_name    0
description          0
brand_category       0
details              0
product_color_id     0
attribute_name       0
attribute_value      0
dtype: int64

In [149]:
pattern['details'] = pattern['details'].str.replace("\n", "")

### Preprocessing

In [150]:
import string 
# define a function to remove punctuation
def removePunctuation(text, punctuations=string.punctuation+"``"+"’"+"”"):
    words=nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in punctuations]
    cleanedText = " ".join(newWords)
    return cleanedText

nltk_stopwords = set(stopwords.words("English"))

# define a function to remove stopwords
def removeStopwords(text, stopwords=nltk_stopwords):
    words = nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in stopwords]
    cleanedText = " ".join(newWords)
    return cleanedText

# define a function to lemmatize all texts
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatizedWords = [lemmatizer.lemmatize(word.lower()) for word in words]
    lemmatizedText = " ".join(lemmatizedWords)
    return lemmatizedText

columns = ["brand", "product_full_name", "description", "details"]
for col in columns: 
    pattern[col] = pattern[col].apply(removePunctuation)
    pattern[col] = pattern[col].apply(removeStopwords)
    pattern[col] = pattern[col].apply(lemmatize)

# generate dummy variables for all different attribute value levels
dummies = pd.get_dummies(pattern['attribute_value'])
pattern = pd.concat([pattern, dummies], axis = 1)
pattern.reset_index(inplace = True)
pattern = pattern.drop('index', axis = 1)

### Models

In [151]:
pattern["input_doc"] = pattern.brand + " " + pattern.product_full_name + " " + pattern.description + " " + pattern.details 

In [152]:
pattern.head()

,product_id,brand,product_full_name,description,brand_category,details,product_color_id,attribute_name,attribute_value,abstract,...,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical,input_doc
0,01DTJ8DQ5SQYBS5HX6TDH2BSJP,hunting season,leather-paneled raffia shoulder bag,handwoven local artisan hunting season 's shou...,women:BAGS:SHOULDER_BAGS,drawstring topcomposition leather raffia plata...,01DTJ8DS7T1QXA88MF9T17ZKS9,pattern,colorblock,0,...,0,0,0,0,0,0,0,0,0,hunting season leather-paneled raffia shoulder...
1,01DTJ8DQ5SQYBS5HX6TDH2BSJP,hunting season,leather-paneled raffia shoulder bag,handwoven local artisan hunting season 's shou...,women:BAGS:SHOULDER_BAGS,drawstring topcomposition leather raffia plata...,01DTJ8DQ64FXG2PTNDZX5QHX4D,pattern,colorblock,0,...,0,0,0,0,0,0,0,0,0,hunting season leather-paneled raffia shoulder...
2,01DTATGG2XJ7QTQC6A4PHQRYW5,alexis,calixte pussy-bow chiffon top,alexis 'calixte top vintage '60s feel somehow ...,women:CLOTHING:TOPS,tie fastening neckcomposition 100 polyesterdry...,01DTATGG356E9CF3V4TWB705BC,pattern,dots,0,...,0,0,0,0,0,0,0,0,0,alexis calixte pussy-bow chiffon top alexis 'c...
3,01DTATGSMSV7FJYTPVAYEHPD8Z,derek lam,belted lace-trimmed printed silk-crepe maxi dress,multicolored silk-crepe slip 100 silk lining 1...,Clothing / Dresses / Maxi,fit true size take normal size petite frame ma...,01DTATGSN4RWS0SJYJJCWR8PA6,pattern,geometric,0,...,0,0,0,0,0,0,0,0,0,derek lam belted lace-trimmed printed silk-cre...
4,01DVCRRBK29DWT82DRBVVY544X,mother pearl,name,black white lyocell partially concealed snap f...,Clothing / Tops / Blouses,fit true size take normal size cut relaxed fit...,01DVCRRBKF3E3MTGQR5NYADG5N,pattern,geometric,0,...,0,0,0,0,0,0,0,0,0,mother pearl name black white lyocell partiall...


### Demo modeling on 'abstract'

In [153]:
X = pattern.iloc[:, -1].values
y = pattern.loc[:, "abstract"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify = y)

In [154]:
docs = list(X_train)
labels = y_train

In [155]:
# set the vocabulary size to some number that represents the total number of unique words in your vocabulary
vocab_size = 5000

In [156]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
tokenizer.word_index

{'size': 1,
 'true': 2,
 '2': 3,
 'fit': 4,
 '5': 5,
 '1': 6,
 '4': 7,
 '8': 8,
 "'s": 9,
 'length': 10,
 'sleeve': 11,
 's': 12,
 '6': 13,
 '10': 14,
 'leather': 15,
 'cotton': 16,
 'style': 17,
 'button': 18,
 'print': 19,
 'waist': 20,
 '0': 21,
 'model': 22,
 'l': 23,
 'wash': 24,
 'small': 25,
 'm': 26,
 'xs': 27,
 'top': 28,
 '100': 29,
 '12': 30,
 'silk': 31,
 'dress': 32,
 'x': 33,
 'dry': 34,
 'wearing': 35,
 'detail': 36,
 'unknown': 37,
 'token': 38,
 '14': 39,
 'body': 40,
 'bust': 41,
 'front': 42,
 'high': 43,
 '24': 44,
 'shirt': 45,
 'xl': 46,
 'hip': 47,
 'rail': 48,
 'classic': 49,
 'imported': 50,
 'stripe': 51,
 'height': 52,
 'made': 53,
 'strap': 54,
 'measurement': 55,
 'nike': 56,
 '9': 57,
 'soft': 58,
 'long': 59,
 'tory': 60,
 'shoe': 61,
 'hand': 62,
 'cut': 63,
 'neck': 64,
 'rayon': 65,
 'one': 66,
 '00': 67,
 'back': 68,
 'cold': 69,
 'pocket': 70,
 'care': 71,
 'recommend': 72,
 'measured': 73,
 '32': 74,
 'bleach': 75,
 'burch': 76,
 'lining': 77,
 'lin

In [158]:
from keras.preprocessing.text import text_to_word_sequence

In [159]:
from typing import List
def integer_encode_documents(docs: List[str], tokenizer: Tokenizer)-> List[List[int]]:
    documents = []
    for d in docs:
        doc_integers = []
        for i in text_to_word_sequence(d):
            doc_integers.append(tokenizer.word_index[i])
        documents.append(doc_integers)
    return documents

In [160]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [161]:
# integer encode the documents
encoded_docs = integer_encode_documents(docs, tokenizer)

from pprint import pprint
pprint(encoded_docs)

[[60,
  76,
  679,
  2062,
  79,
  15,
  184,
  184,
  226,
  262,
  9,
  92,
  313,
  1252,
  1720,
  135,
  2062,
  466,
  186,
  60,
  9,
  266,
  208,
  111,
  79,
  27,
  252,
  12,
  193,
  26,
  90,
  23,
  369,
  46,
  281,
  933,
  934,
  1017,
  6,
  184,
  557,
  6,
  6,
  7,
  370,
  2848],
 [56,
  49,
  1434,
  680,
  381,
  86,
  270,
  221,
  198,
  227,
  2063,
  6,
  558,
  411,
  1435,
  161,
  381,
  178,
  116,
  42,
  56,
  2849,
  1,
  2064,
  2850,
  1,
  2851,
  1253,
  2852,
  66,
  935,
  2064,
  314,
  2853,
  2854,
  4,
  816,
  2855,
  1,
  4,
  641,
  50,
  4,
  2,
  1,
  93,
  97,
  1,
  187,
  147],
 [56,
  124,
  303,
  1436,
  2065,
  86,
  681,
  106,
  315,
  864,
  139,
  1721,
  682,
  2066,
  1104,
  1254,
  86,
  1018,
  443,
  304,
  1437,
  683,
  2,
  1],
 [684,
  642,
  685,
  643,
  150,
  510,
  130,
  123,
  64,
  130,
  2067,
  150,
  510,
  175,
  16,
  241,
  2856,
  1105,
  2,
  1,
  27,
  21,
  12,
  3,
  7,
  26,
  13,
  8,
  23,
  1

  11,
  1031,
  1283,
  104,
  452,
  185,
  2111,
  725,
  2112,
  68,
  172,
  2,
  1,
  27,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39,
  149,
  10,
  1,
  142,
  42,
  18,
  939,
  64,
  2113,
  2907,
  769,
  11,
  66,
  18,
  828,
  400,
  419,
  189],
 [48,
  497,
  37,
  38,
  36,
  58,
  175,
  88,
  11,
  123,
  64,
  180,
  28,
  170,
  1284,
  1135,
  313,
  49,
  161,
  95,
  655,
  1136,
  19,
  264,
  433,
  234,
  78,
  168,
  449,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  44,
  6,
  3,
  73,
  25,
  22,
  9,
  55,
  52,
  5,
  98,
  41,
  74,
  20,
  44,
  47,
  90],
 [656,
  2114,
  293,
  28,
  1285,
  1286,
  1468,
  1760,
  2115,
  323,
  302,
  19,
  318,
  199,
  45,
  248,
  830,
  196,
  16,
  2,
  1,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  39,
  102,
  6,
  3,
  10,
  1,
  142,
  42,
  18,
  1029,
  1469,
  2908,
  516,
  2909,
  16,
  3,
  1137,
  82,
  249,
  24,
  15

  2,
  1,
  40,
  10,
  44,
  6,
  3,
  73,
  25,
  22,
  9,
  55,
  52,
  5,
  98,
  41,
  74,
  20,
  44,
  47,
  90],
 [457,
  2155,
  2155,
  627,
  53,
  58,
  964,
  100,
  355,
  66,
  413,
  1501,
  2156,
  354,
  151,
  99,
  127,
  123,
  64,
  953,
  1782,
  184,
  721,
  1151,
  544,
  1502,
  1503,
  125,
  698,
  19,
  22,
  889,
  458,
  5,
  8,
  344,
  109,
  1,
  12,
  40,
  29,
  964,
  77,
  29,
  167,
  36,
  29,
  602,
  18,
  1152,
  545,
  627,
  125,
  1783,
  24,
  307,
  890,
  1504,
  1505,
  546,
  158,
  1506,
  127,
  547,
  839,
  875,
  34,
  82,
  501,
  502,
  1153],
 [439,
  1281,
  150,
  2157,
  129,
  2158,
  387,
  1507,
  289,
  150,
  781,
  129,
  937,
  531,
  378,
  2157,
  1508,
  1467,
  475,
  1028,
  68,
  2,
  1,
  80,
  21,
  27,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39],
 [891,
  1304,
  1509,
  1510,
  2969,
  244,
  277,
  1511,
  200,
  697,
  1305,
  965,
  35,
  244,
  277,
  200,
  966,
  350,
  9,
  2970,
 

  201,
  250,
  290,
  647,
  113,
  99,
  1530,
  15,
  1531,
  89,
  551,
  205,
  584,
  89,
  198,
  227,
  1174,
  3,
  1815,
  397,
  201,
  250,
  724,
  1532,
  198,
  982,
  3,
  983,
  17,
  4,
  2,
  165,
  160,
  155,
  97,
  1],
 [56,
  124,
  303,
  1523,
  3023,
  86,
  3024,
  2188,
  24,
  106,
  2189,
  878,
  315,
  266,
  124,
  303,
  1523,
  137,
  86,
  1033,
  1796,
  303,
  124,
  285,
  215,
  25,
  122,
  6,
  3,
  1,
  116,
  178,
  116,
  971,
  410,
  1163,
  896,
  843,
  1797,
  303,
  124,
  600,
  89,
  392,
  628,
  124,
  221,
  1798,
  342,
  94,
  77,
  270,
  128,
  192,
  9,
  61],
 [283,
  3025,
  1321,
  218,
  28,
  1816,
  218,
  2135,
  78,
  1533,
  129,
  339,
  984,
  521,
  11,
  166,
  393,
  417,
  2,
  1,
  80,
  67,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39],
 [60,
  76,
  1175,
  506,
  631,
  137,
  506,
  631,
  1176,
  79,
  251,
  773,
  2190,
  3026,
  17,
  3027,
  2191,
  19,
  106,
  319,
  1,

  5],
 [60,
  76,
  679,
  616,
  105,
  9,
  589,
  124,
  113,
  1255,
  60,
  271,
  759,
  274,
  105,
  360,
  346,
  54,
  256,
  148,
  4,
  2,
  1],
 [60,
  76,
  1193,
  51,
  584,
  1159,
  31,
  565,
  129,
  31,
  45,
  339,
  3081,
  565,
  659,
  1540,
  797,
  584,
  1159,
  834,
  851,
  607,
  1840,
  296,
  1,
  122,
  66,
  1,
  288,
  147,
  233,
  1,
  314],
 [216,
  323,
  302,
  19,
  16,
  579,
  88,
  355,
  209,
  20,
  88,
  63,
  323,
  302,
  16,
  579,
  148,
  824,
  655,
  1841,
  668,
  2,
  1],
 [56,
  1170,
  3082,
  1478,
  86,
  3083,
  19,
  3084,
  3085,
  995,
  3086,
  137,
  86,
  2230,
  3087,
  1194,
  630,
  2231,
  355,
  1823,
  2,
  1,
  748,
  798,
  748,
  61,
  739,
  61,
  1195,
  1196,
  481,
  1067,
  9,
  147],
 [651,
  652,
  3088,
  123,
  64,
  837,
  178,
  435,
  108,
  64,
  68,
  19,
  837,
  203,
  4,
  2,
  1,
  27,
  3,
  12,
  7,
  26,
  13,
  23,
  8],
 [56,
  552,
  33,
  1068,
  1842,
  86,
  798,
  172,
  978,
  1332

...

  49,
  331,
  237,
  53,
  916,
  638,
  1715,
  2055,
  19,
  2,
  1],
 [48,
  195,
  37,
  38,
  36,
  58,
  78,
  59,
  11,
  18,
  28,
  114,
  460,
  114,
  107,
  70,
  225,
  78,
  194,
  65,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  149,
  6,
  7,
  73,
  25],
 [171,
  140,
  2809,
  159,
  492,
  277,
  15,
  1495,
  2809,
  759,
  461,
  113,
  159,
  2472,
  265,
  568,
  636,
  123,
  63,
  113,
  492,
  277,
  15,
  425,
  313,
  37,
  38],
 [329,
  51,
  620,
  45,
  209,
  20,
  170,
  338,
  1664,
  1779,
  4509,
  163,
  88,
  11,
  45,
  853,
  805,
  1204,
  280,
  2,
  1,
  511,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30],
 [329,
  635,
  370,
  64,
  513,
  196,
  120,
  32,
  63,
  383,
  53,
  1381,
  196,
  120,
  1412,
  32,
  264,
  406,
  2359,
  57,
  376,
  5,
  92,
  101,
  329,
  543,
  589,
  122,
  66,
  1,
  1,
  122,
  66,
  1],
 [2630,
  2631,
  1695,
  2632,
  352,
  2633,
  9,
  1695,
  

  6,
  117,
  7,
  466,
  4558],
 [60,
  76,
  679,
  751,
  429,
  15,
  239,
  1621,
  751,
  1386,
  1921,
  695,
  99,
  238,
  207,
  239,
  1622,
  504,
  341,
  232,
  15,
  54,
  247,
  136,
  33,
  14,
  143,
  33,
  5,
  613,
  145,
  162,
  183,
  223,
  57,
  54,
  146,
  3,
  21,
  188],
 [1917,
  1265,
  477,
  346,
  1156,
  196,
  4559,
  1808,
  131,
  424,
  161,
  196,
  2813,
  702,
  992,
  2813,
  14,
  2354,
  249,
  24,
  4,
  2,
  1,
  93,
  97,
  1,
  100,
  2764,
  926,
  423,
  612,
  444,
  253,
  96,
  22,
  372,
  210,
  35,
  1,
  25],
 [48,
  1304,
  37,
  38,
  36,
  88,
  11,
  778,
  391,
  78,
  141,
  151,
  627,
  123,
  220,
  703,
  184,
  139,
  70,
  161,
  78,
  1297,
  954,
  51,
  354,
  4,
  627,
  1416,
  207,
  17,
  272,
  109,
  203,
  66,
  232,
  427,
  92,
  417,
  68,
  1057,
  225,
  65,
  194,
  78,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  42,
  40,
  10,
  2814,
  647,
  20,
  301,
  469,
  6

  435,
  476,
  63,
  622,
  2534,
  1227,
  15,
  263,
  136,
  33,
  5,
  143,
  33,
  6],
 [295,
  1012,
  51,
  644,
  11,
  16,
  28,
  51,
  1013,
  17,
  108,
  560,
  16,
  28,
  2671,
  1705,
  406,
  92,
  2,
  1,
  80,
  67,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39],
 [2249,
  2823,
  714,
  2251,
  1344,
  312,
  16,
  156,
  29,
  312,
  16,
  62,
  24,
  34,
  82,
  276,
  106,
  714,
  1574,
  2252,
  50,
  4,
  2,
  1,
  93,
  97,
  1,
  100,
  181,
  4,
  63,
  452,
  185,
  199,
  1856,
  423,
  1575,
  1576,
  5,
  263,
  35,
  1,
  12,
  26],
 [2772,
  2773,
  352,
  1142,
  2774,
  186,
  1817,
  222,
  352,
  339,
  127,
  2775,
  89,
  599,
  370,
  63,
  1228,
  548,
  113,
  215,
  25,
  122,
  6,
  3,
  1],
 [283,
  2625,
  28,
  746,
  180,
  129,
  409,
  238,
  1626,
  92,
  507,
  901,
  4610,
  389,
  87,
  825,
  924,
  11,
  2,
  1,
  80,
  67,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39],
 

  765,
  495,
  381,
  416,
  91,
  136,
  33,
  263,
  91,
  143,
  33,
  5,
  91,
  145,
  162,
  183,
  338,
  223,
  57,
  54,
  146,
  6,
  7,
  188],
 [283,
  4667,
  913,
  11,
  28,
  697,
  1131,
  4668,
  343,
  2266,
  505,
  180,
  129,
  847,
  913,
  11,
  947,
  2751,
  2737,
  1171,
  2,
  1,
  80,
  67,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39,
  44,
  4669,
  68,
  84,
  366,
  68,
  595,
  18,
  232,
  308,
  958,
  4670,
  453,
  580,
  414,
  82,
  249,
  24,
  158,
  731],
 [60,
  76,
  1345,
  58,
  429,
  1702,
  15,
  87,
  110,
  429,
  1702,
  36,
  695,
  905,
  321,
  110,
  853,
  620,
  341,
  87,
  54,
  373,
  132,
  208,
  111,
  79,
  14,
  91,
  136,
  33,
  98,
  143,
  33,
  7,
  145,
  162,
  183,
  25,
  30,
  91,
  126,
  91,
  359,
  54,
  146,
  6,
  13,
  188],
 [911,
  2827,
  2828,
  43,
  20,
  396,
  317,
  4671,
  2827,
  1364,
  106,
  768,
  396,
  53,
  132,
  2828,
  2595,
  406,
  752,
  196,
  286,


  7,
  26,
  13,
  8,
  23,
  14],
 [860,
  1087,
  150,
  19,
  31,
  119,
  45,
  150,
  19,
  31,
  119,
  18,
  182,
  42,
  2060,
  225,
  31,
  194,
  231,
  2061,
  29,
  31,
  34,
  82,
  53,
  379,
  4,
  2,
  1,
  93,
  97,
  1,
  2826,
  87,
  974,
  1052,
  93,
  278,
  1,
  63,
  166,
  999,
  529,
  253,
  371,
  372,
  210,
  35,
  479,
  169],
 [1911,
  1912,
  1913,
  1914,
  199,
  16,
  119,
  316,
  175,
  275,
  1344,
  131,
  16,
  119,
  316,
  175,
  156,
  796,
  16,
  6,
  730,
  249,
  24,
  53,
  1915,
  4,
  2,
  1,
  93,
  97,
  1,
  63,
  108,
  1619,
  2681,
  2682,
  423,
  253,
  371,
  269,
  5,
  57,
  35,
  1,
  25],
 [48,
  1771,
  37,
  38,
  36,
  59,
  11,
  29,
  16,
  208,
  1292,
  18,
  85,
  28,
  754,
  18,
  137,
  259,
  422,
  207,
  49,
  884,
  885,
  1036,
  1293,
  692,
  835,
  29,
  16,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  149,
  73,
  25,
  22,
  9,
  55,
  52,
  5,
  98,
  41,

In [162]:
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))

In [163]:
# get the max length in terms of token length
max_length = get_max_token_length_per_doc(docs)

In [164]:
max_length

150

In [165]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_docs)

Padded docs: [[  60   76  679 ...    0    0    0]
 [  56   49 1434 ...    0    0    0]
 [  56  124  303 ...    0    0    0]
 ...
 [  56  124 1683 ...    0    0    0]
 [  48 4796   37 ...    0    0    0]
 [ 216  486   19 ...    0    0    0]]


In [166]:
EMBEDDING_SIZE = 100

### Define deep learning model

In [167]:
# define the model

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
model.add(Flatten()) 

model.add(Dense(1, activation='sigmoid')) 

In [168]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()

Model: "sequential_174"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_174 (Embedding)    (None, 150, 100)          500000    
_________________________________________________________________
flatten_174 (Flatten)        (None, 15000)             0         
_________________________________________________________________
dense_174 (Dense)            (None, 1)                 15001     
Total params: 515,001
Trainable params: 515,001
Non-trainable params: 0
_________________________________________________________________


In [169]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Accuracy: 99.834573


In [170]:
embedding_layer = model.layers[0]
embedding_layer.get_weights()[0].shape

(5000, 100)

In [171]:
test_docs = list(X_test)
test_labels = y_test

In [172]:
encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

# pad test documents
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_test_docs)

Padded docs: [[ 177  173   28 ...    0    0    0]
 [ 911  176   43 ...    0    0    0]
 [ 177  173   95 ...    0    0    0]
 ...
 [ 656 2998  244 ...    0    0    0]
 [ 264  103   19 ...    0    0    0]
 [ 494  953  762 ...    0    0    0]]


In [173]:
loss, accuracy = model.evaluate(padded_test_docs, test_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 96.526057


In [174]:
pattern.iloc[:, 29]

0       hunting season leather-paneled raffia shoulder...
1       hunting season leather-paneled raffia shoulder...
2       alexis calixte pussy-bow chiffon top alexis 'c...
3       derek lam belted lace-trimmed printed silk-cre...
4       mother pearl name black white lyocell partiall...
                              ...                        
1607    ulla johnson natalia metallic shibori long sle...
1608    zadig voltaire paros crystal buckle ankle stra...
1609    zadig voltaire used sneaker graffiti graphic c...
1610    zadig voltaire rock star circus metallic leath...
1611    sarto franco sarto bordo slide sandal blunted ...
Name: input_doc, Length: 1612, dtype: object

### For loop to test all attribute values based on the deep learning model

In [ ]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [ ]:
def get_max_token_length_per_doc(docs):
    return max(list(map(lambda x: len(x.split()), docs)))

In [183]:
X = pattern.iloc[:, -1].values
pattern_values = list(pattern.columns.values[9:29])
max_length_list = []
for i in range(9, 29):
    y = pattern.iloc[:, i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify = y)

    docs = list(X_train)
    labels = y_train

    vocab_size = 6000
    EMBEDDING_SIZE = 100

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(docs)
    tokenizer.word_index

    encoded_docs = integer_encode_documents(docs, tokenizer)

    max_length = get_max_token_length_per_doc(docs)

    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
    model.add(Flatten()) 

    model.add(Dense(1, activation='sigmoid')) 

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    # fit the model
    model.fit(padded_docs, labels, epochs=50, verbose=0)
    
    #save model
    model.save("{}_model.h5".format(pattern_values[i-9]))

    embedding_layer = model.layers[0]
    embedding_layer.get_weights()[0].shape

    test_docs = list(X_test)
    test_labels = y_test

    encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

    # pad test documents
    padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

    loss, accuracy = model.evaluate(padded_test_docs, test_labels, verbose=0) # the average accuracy is around 95%
    #save tokenizer
    with open('{}_tokenizer.pickle'.format(pattern_values[i-9]), 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    max_length_list.append(max_length)

D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of

In [181]:
max_length_list

[150,
 150,
 124,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150]

# Apply our trained model on full data

In [108]:
full_data = pd.read_csv('full_data_final version.csv')

In [109]:
#replace null values with UNKNOWN_TOKEN
full_data['description'] = full_data['description'].fillna('UNKNOWN_TOKEN')
full_data['details'] = full_data['details'].fillna('UNKNOWN_TOKEN')
full_data['brand_category'] = full_data['brand_category'].fillna('UNKNOWN_TOKEN')
full_data['details'] = full_data['details'].str.replace("\n", "")

In [110]:
import string 
# define a function to remove punctuation
def removePunctuation(text, punctuations=string.punctuation+"``"+"’"+"”"):
    words=nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in punctuations]
    cleanedText = " ".join(newWords)
    return cleanedText

nltk_stopwords = set(stopwords.words("English"))

# define a function to remove stopwords
def removeStopwords(text, stopwords=nltk_stopwords):
    words = nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in stopwords]
    cleanedText = " ".join(newWords)
    return cleanedText

# define a function to lemmatize all texts
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatizedWords = [lemmatizer.lemmatize(word.lower()) for word in words]
    lemmatizedText = " ".join(lemmatizedWords)
    return lemmatizedText

columns = ["brand", "product_full_name", "description", "details"]
for col in columns: 
    full_data[col] = full_data[col].apply(removePunctuation)
    full_data[col] = full_data[col].apply(removeStopwords)
    full_data[col] = full_data[col].apply(lemmatize)

In [111]:
full_data['abstract'] = 0
full_data['animal'] = 0
full_data['camouflage'] = 0
full_data['colorblock'] = 0
full_data['dots'] = 0
full_data['floral'] = 0
full_data['geometric'] = 0
full_data['graphic'] = 0
full_data['houndstooth'] = 0
full_data['logo'] = 0
full_data['monogram'] = 0
full_data['multiprint'] = 0
full_data['paisley'] = 0
full_data['pinstripe'] = 0
full_data['plaid'] = 0
full_data['stripe'] = 0
full_data['stripehorizontal'] = 0
full_data['stripevertical'] = 0
full_data['tiedye'] = 0
full_data['tropical'] = 0
full_data["input_doc"] = full_data.brand+" "+full_data.product_full_name+" "+full_data.description+" "+full_data.details 

In [112]:
full_data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical,input_doc
0,01DSE9TC2DQXDG6GWKW9NMJ416,banana republic,514683,ankle-strap pump,modern pump rounded silhouette ankle strap ext...,Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0,0,0,0,0,0,0,0,0,banana republic ankle-strap pump modern pump r...
1,01DSE9SKM19XNA6SJP36JZC065,banana republic,526676,petite tie-neck top,dress jean sneaker dress tailored trouser heel...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0,0,0,0,0,0,0,0,0,banana republic petite tie-neck top dress jean...
2,01DSJX8GD4DSAP76SPR85HRCMN,loewe,4.001E+11,52mm padded leather round sunglass,padded leather cover classic round sunglass,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,...,0,0,0,0,0,0,0,0,0,loewe 52mm padded leather round sunglass padde...
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,converse,4.00012E+11,baby 's little kid 's all-star two-tone mid-to...,iconic mid-top design get added dose support p...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,...,0,0,0,0,0,0,0,0,0,converse baby 's little kid 's all-star two-to...
4,01DSK15ZD4D5A0QXA8NSD25YXE,alexander mcqueen,4.00011E+11,64mm rimless sunglass,hexagonal shade offer rimless view intricate n...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,...,0,0,0,0,0,0,0,0,0,alexander mcqueen 64mm rimless sunglass hexago...


In [113]:
full_data.shape

(48979, 34)

In [117]:
for i in range(13, 33):
    X_train = pattern.iloc[:, -1].values
    y_train = pattern.iloc[:, i-4].values
    X_test = full_data.iloc[:, -1].values

    docs = list(X_train)
    labels = y_train

    vocab_size = 6000

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(docs)
    tokenizer.word_index

    encoded_docs = integer_encode_documents(docs, tokenizer)

    max_length = get_max_token_length_per_doc(docs)

    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
    model.add(Flatten()) 

    model.add(Dense(1, activation='sigmoid')) 

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    # fit the model
    model.fit(padded_docs, labels, epochs=50, verbose=0)

    embedding_layer = model.layers[0]
    embedding_layer.get_weights()[0].shape

    test_docs = list(X_test)
    
    encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

    # pad test documents
    padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
    prediction = model.predict(padded_test_docs, verbose=0)
    full_data.iloc[:, i] = prediction

D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of

In [118]:
full_data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical,input_doc
0,01DSE9TC2DQXDG6GWKW9NMJ416,banana republic,514683,ankle-strap pump,modern pump rounded silhouette ankle strap ext...,Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0.000030,0.000282,0.000850,0.000016,0.000034,0.000016,0.000494,0.000018,0.000122,banana republic ankle-strap pump modern pump r...
1,01DSE9SKM19XNA6SJP36JZC065,banana republic,526676,petite tie-neck top,dress jean sneaker dress tailored trouser heel...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0.000098,0.000837,0.000153,0.000070,0.000758,0.000452,0.000069,0.001047,0.000127,banana republic petite tie-neck top dress jean...
2,01DSJX8GD4DSAP76SPR85HRCMN,loewe,4.001E+11,52mm padded leather round sunglass,padded leather cover classic round sunglass,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,...,0.000155,0.000459,0.000281,0.000942,0.001336,0.000279,0.002808,0.000308,0.000730,loewe 52mm padded leather round sunglass padde...
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,converse,4.00012E+11,baby 's little kid 's all-star two-tone mid-to...,iconic mid-top design get added dose support p...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,...,0.000122,0.000458,0.000258,0.000083,0.001218,0.000254,0.000367,0.000197,0.003240,converse baby 's little kid 's all-star two-to...
4,01DSK15ZD4D5A0QXA8NSD25YXE,alexander mcqueen,4.00011E+11,64mm rimless sunglass,hexagonal shade offer rimless view intricate n...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,...,0.000086,0.000542,0.000325,0.002392,0.000698,0.000322,0.002572,0.000754,0.000894,alexander mcqueen 64mm rimless sunglass hexago...


In [128]:
df1 = full_data.iloc[:, 13:33]
df1.head()

,abstract,animal,camouflage,colorblock,dots,floral,geometric,graphic,houndstooth,logo,monogram,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical
0,0.000157,0.833238,0.000109,0.001627,0.007265,0.003377,0.000841,1.817074e-05,0.000027,0.001230,0.000483,0.000030,0.000282,0.000850,0.000016,0.000034,0.000016,0.000494,0.000018,0.000122
1,0.000280,0.026494,0.000024,0.000001,0.005849,0.415962,0.017263,2.250702e-07,0.000017,0.000026,0.000242,0.000098,0.000837,0.000153,0.000070,0.000758,0.000452,0.000069,0.001047,0.000127
2,0.001264,0.022300,0.000102,0.002622,0.002695,0.000675,0.007148,1.168720e-03,0.000435,0.259638,0.000218,0.000155,0.000459,0.000281,0.000942,0.001336,0.000279,0.002808,0.000308,0.000730
3,0.001192,0.001265,0.000047,0.001412,0.000304,0.002869,0.027290,1.037046e-04,0.000051,0.066059,0.000094,0.000122,0.000458,0.000258,0.000083,0.001218,0.000254,0.000367,0.000197,0.003240
4,0.001178,0.030480,0.000149,0.003791,0.000922,0.000675,0.000291,7.925456e-04,0.000172,0.036569,0.000161,0.000086,0.000542,0.000325,0.002392,0.000698,0.000322,0.002572,0.000754,0.000894


In [129]:
df1['patterns/prints'] = df1.idxmax(axis = 1)

In [131]:
full_data['patterns/prints'] = df1['patterns/prints']